https://towardsdatascience.com/keyword-extraction-python-tf-idf-textrank-topicrank-yake-bert-7405d51cd839

In [ ]:
!pip install trafilatura

In [ ]:
!pip install summa

In [ ]:
!pip install git+https://github.com/LIAAD/yake

In [ ]:
!pip install keyBERT

In [4]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sdeshpande/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sdeshpande/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/sdeshpande/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
#We extract the articles' texts from each webpage and perform some basic text cleaning. We take only the first 5000 characters.
import trafilatura
array_links = [
    "https://en.wikipedia.org/wiki/Coronavirus_disease_2019", 
    "https://en.wikipedia.org/wiki/Recession", 
    "https://en.wikipedia.org/wiki/Vienna", 
    "https://en.wikipedia.org/wiki/Machine_learning", 
    "https://en.wikipedia.org/wiki/Graph_database"
]
array_text = []
for l in array_links:
    html = trafilatura.fetch_url(l)
    text = trafilatura.extract(html)
    text_clean = text.replace("\n", " ").replace("\'", "")
    array_text.append(text_clean[0:5000])

# Keywords Extraction with TF-IDF

In [ ]:
from itertools import islice
from tqdm.notebook import tqdm
from re import sub

In [ ]:
num_lines = sum(1 for line in open("/Users/sdeshpande/Desktop/text_analysis_scripts/SmartDataAnalytics-Wikipedia_TF_IDF_Dataset-a614a46/en/wiki_tfidf_terms.csv"))
with open("/Users/sdeshpande/Desktop/text_analysis_scripts/SmartDataAnalytics-Wikipedia_TF_IDF_Dataset-a614a46/en/wiki_tfidf_terms.csv") as file:
    dict_idf = {}
    with tqdm(total=num_lines) as pbar:
        for i, line in tqdm(islice(enumerate(file), 1, None)):
            try: 
                cells = line.split(",")
                idf = float(sub("[^0-9.]", "", cells[3]))
                dict_idf[cells[0]] = idf
            except: 
                print("Error on: " + line)
            finally:
                pbar.update(1)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from numpy import array, log
vectorizer = CountVectorizer()
tf = vectorizer.fit_transform([x.lower() for x in array_text])
tf = tf.toarray()
tf = log(tf + 1)

In [ ]:
tfidf = tf.copy()
words = array(vectorizer.get_feature_names())
for k in tqdm(dict_idf.keys()):
    if k in words:
        tfidf[:, words == k] = tfidf[:, words == k] * dict_idf[k]
    pbar.update(1)


In [ ]:
for j in range(tfidf.shape[0]):
    print("Keywords of article", str(j+1), words[tfidf[j, :].argsort()[-5:][::-1]])

# Keywords Extraction with TextRank

In [1]:
from summa import keywords

In [5]:
for j in range(len(array_text)):
    print("Keywords of article", str(j+1), "\n", (keywords.keywords(array_text[j], words=5)).split("\n"))

Keywords of article 1 
 ['symptoms', 'disease', 'covid', 'syndrome', 'fever']
Keywords of article 2 
 ['recession', 'recessions', 'economics', 'economic', 'policies', 'policy', 'governments', 'government', 'shapes', 'shape']
Keywords of article 3 
 ['vienna', 'viennas', 'city', 'cities', 'citys', 'mi', 'rank', 'ranked', 'cultural', 'culture']
Keywords of article 4 
 ['data', 'algorithms', 'algorithm', 'tasks', 'task', 'computational', 'computers', 'computing', 'problems', 'problem']
Keywords of article 5 
 ['graphs', 'graph database', 'databases', 'model', 'models', 'data', 'relates', 'relational', 'relation']


# Keywords Extraction with YAKE!

In [8]:
from yake import KeywordExtractor

In [9]:
kw_extractor = KeywordExtractor(lan="en", n=1, top=5)
for j in range(len(array_text)):
    keywords = kw_extractor.extract_keywords(text=array_text[j])
    keywords = [x for x, y in keywords]
    print("Keywords of article", str(j+1), "\n", keywords)

Keywords of article 1 
 ['symptoms', 'pneumonia', 'coronavirus', 'people', 'syndrome']
Keywords of article 2 
 ['economic', 'recession', 'recessions', 'gdp', 'united']
Keywords of article 3 
 ['vienna', 'city', 'state', 'capital', 'german']
Keywords of article 4 
 ['learning', 'machine', 'computer', 'tasks', 'computers']
Keywords of article 5 
 ['graph', 'databases', 'database', 'data', 'relationships']


# Keywords Extraction with BERT

In [10]:
from keybert import KeyBERT
kw_extractor = KeyBERT('distilbert-base-nli-mean-tokens')
for j in range(len(array_text)):
    keywords = kw_extractor.extract_keywords(array_text[j], keyphrase_length=1, stop_words='english')
    print("Keywords of article", str(j+1), "\n", keywords)

Keywords of article 1 
 ['coronavirus', 'pneumonia', 'vaccines', 'virus', '2019']
Keywords of article 2 
 ['recessions', 'recession', 'unemployment', 'pandemic', 'worsening']
Keywords of article 3 
 ['austrias', 'vienna', 'austria', 'austrian', 'viennas']
Keywords of article 4 
 ['algorithms', 'algorithm', 'computational', 'computers', 'mathematical']
Keywords of article 5 
 ['graphs', 'graph', 'databases', 'web', 'database']
